In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from eumap.misc import find_files, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.mapper import SpaceOverlay
import warnings
import matplotlib


df = pd.read_parquet(f'/home/xuemeng/work_xuemeng/soc/SoilHealthDataCube/soil_property_model_pipeline/soil_overlaid.pq')
print(df.shape)

(409369, 639)


### convert categorical variables to numerical using dummy

In [15]:
# lithology = '''1	alkaliOlivineBasalt
# 2	amphibolite
# 3	andesite
# 4	anorthositicRock
# 5	arenite
# 6	ashBrecciaBombOrBlockTephra
# 7	ashTuffLapillistoneAndLapilliTuff
# 8	basalt
# 9	basanite
# 10	biogenicSilicaSedimentaryRock
# 11	breccia
# 12	brecciaGougeSeries
# 13	carbonateMud
# 14	carbonateOoze
# 15	carbonateSedimentaryRock
# 16	carbonatite
# 17	chalk
# 18	chloriteActinoliteEpidoteMetamorphicRock
# 19	clasticSandstone
# 20	clasticSediment
# 21	clasticSedimentaryMaterial
# 22	clasticSedimentaryRock
# 23	clay
# 24	claystone
# 25	conglomerate
# 26	dacite
# 27	diamictite
# 28	diamicton
# 29	diorite
# 30	dioriticRock
# 31	doleriticRock
# 32	dolomite
# 33	dolomiticOrMagnesianSedimentaryRock
# 34	eclogite
# 35	exoticCompositionIgneousRock
# 36	fineGrainedIgneousRock
# 37	foidBearingSyenite
# 38	foidSyenitoid
# 39	foliatedMetamorphicRock
# 40	fragmentalIgneousMaterial
# 41	gabbro
# 42	gabbroicRock
# 43	gabbroid
# 44	glaucophaneLawsoniteEpidoteMetamorphicRock
# 45	gneiss
# 46	granite
# 47	granitoid
# 48	granodiorite
# 49	granulite
# 50	gravel
# 51	gypsumOrAnhydrite
# 52	hornfels
# 53	igneousMaterial
# 54	igneousRock
# 55	impactGeneratedMaterial
# 56	impureCarbonateSediment
# 57	impureCarbonateSedimentaryRock
# 58	impureDolomite
# 59	impureLimestone
# 60	komatiiticRock
# 61	limestone
# 62	marble
# 63	metamorphicRock
# 64	metasomaticRock
# 65	micaSchist
# 66	migmatite
# 67	monzogranite
# 68	monzonite
# 69	mud
# 70	mudstone
# 71	myloniticRock
# 72	nonClasticSiliceousSedimentaryRock
# 73	organicBearingMudstone
# 74	organicRichSediment
# 75	organicRichSedimentaryMaterial
# 76	orthogneiss
# 77	paragneiss
# 78	peat
# 79	peridotite
# 80	phaneriticIgneousRock
# 81	phonolite
# 82	phyllite
# 83	phyllonite
# 84	porphyry
# 85	pyroclasticMaterial
# 86	pyroclasticRock
# 87	pyroxenite
# 88	quartzDiorite
# 89	quartzite
# 90	residualMaterial
# 91	rhyolite
# 92	rhyolitoid
# 93	rock
# 94	sand
# 95	sandstone
# 96	schist
# 97	sediment
# 98	sedimentaryMaterial
# 99	sedimentaryRock
# 100	serpentinite
# 101	shale
# 102	silicateMud
# 103	silt
# 104	siltstone
# 105	skarn
# 106	slate
# 107	spilite
# 108	syenite
# 109	syeniticRock
# 110	syenogranite
# 111	tephrite
# 112	tholeiiticBasalt
# 113	tonalite
# 114	trachyte
# 115	trachyticRock
# 116	trachytoid
# 117	travertine
# 118	tuffBrecciaAgglomerateOrPyroclasticBreccia
# 119	ultramaficIgneousRock
# 120	wacke
# 121	waste
# 122	Unknown
# 123	Unpopulated'''

# lithology = lithology.split('\n')
# value = ['lithology.' + i.split('\t')[0] + '.lithology_egdi.1m_c_250m_s_20000101_20221231_eu_epsg.3035_v20240530' for i in lithology]
# descp = [i.split('\t')[1] for i in lithology]
# descp = ['lithology_egdi_prob_'+i for i in descp]
# lithology_map = dict(zip(value,descp))

# # df = df.rename(columns=lithology_map)
# # print(df.shape)

In [3]:
# for ii in df.columns:
#     if 'egdi' in ii:
#         print(ii)

### assign spatial blocking id

In [4]:
# create a tiling system first
# from eumap.parallel import TilingProcessing
from pathlib import Path
import rasterio
import geopandas as gpd
from shapely.geometry import box

# # create tiling system
# raster_layer_fn = f'http://192.168.1.30:8333/ai4sh-landmasked/ndvi/ndvi_glad.landast.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127.tif'
# ds = rasterio.open(raster_layer_fn)
# tiles_size = ds.transform[0] * 1000 # 30m -> 30km
# tiling_system = TilingProcessing.generate_tiles(tiles_size, extent=ds.bounds, crs=ds.crs, raster_layer_fn=raster_layer_fn)
# tiling_system = tiling_system.to_crs("EPSG:4326")
# tiling_system = tiling_system[['tile_id','geometry']]
# tiling_system.to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg',  driver="GPKG")
# # tiling_system[tiling_system['raster_mode_count'] > 0].to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg.gpkg',  driver="GPKG")


from shapely.geometry import Point
tiles = gpd.read_file(f'/home/xuemeng/work_xuemeng/soc/SoilHealthDataCube/data/002_eu.tiles_epsg.3035.gpkg')

# dff = pd.read_csv(f'{folder}/data/001_covar_overlayed.csv',low_memory=False)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(tiles.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
print(joined_gdf.shape)

# drop the redundant columns
joined_gdf = joined_gdf.drop(columns=['geometry', 'index_right'])
dff = joined_gdf.rename(columns={'id_right':'tile_id','id_left':'id'})
print(dff.shape)

(409369, 642)
(409369, 640)


### meta data check

In [5]:
for meta_col in ['time', 'lat', 'lon', 'id', 'ref', 'nuts0', 'hzn_dep', 'lc_survey','tile_id']:
    num_nan = dff[meta_col].isna().sum()
    print(f'{num_nan} nan record for {meta_col}')
    
print()
print(dff.shape)

0 nan record for time
0 nan record for lat
0 nan record for lon
0 nan record for id
0 nan record for ref
1002 nan record for nuts0
0 nan record for hzn_dep
339402 nan record for lc_survey
82 nan record for tile_id

(409369, 640)


### prepare covar list

In [6]:
# fix cropland layer
crop_layer = 'cropland.extent_glad.interpolate_p_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20240604'
dff.loc[dff[crop_layer]>0,crop_layer] = 100
dff.loc[dff[crop_layer].isna(),crop_layer] = 0

In [7]:
covs_all = dff.columns.values.tolist()[41:-1]
print(len(covs_all))
for ii in covs_all:
    numnan = dff[ii].isna().sum()
    rtonan = numnan*100/len(dff)
    if rtonan>5:
        print(round(rtonan, 1), numnan, ii)
        covs_all.remove(ii)
        
print(len(covs_all))

598
10.2 41665 accum.ndwi_glad.landsat.seasconv.m.yearly_p50_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20240513
18.8 76898 accum.ndvi_glad.landsat.seasconv.m.yearly_p50_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20240513
89.7 367073 CHELSA_swe_1981-2010_V.2.1
595


In [8]:
## drop accumulation covariates due to the artifacts it introduce
print(len(covs_all))
for ii in covs_all:
    if ('accum' in ii) & ('seasconv' in ii):
        print(ii)
        covs_all.remove(ii)
        
print(len(covs_all))

595
accum.ndvi_glad.landsat.seasconv_m_30m_s_YYYY0701_YYYY0831_eu_epsg.3035_v20240513
accum.ndti_glad.landsat.seasconv.m.yearly_min_30m_s_YYYY0101_YYYY_eu_epsg.3035_v20240513
593


In [9]:
for col in covs_all:
    ini_nan = dff[col].isna().sum()
    if col != 'tile_id':  # Avoid modifying 'tile_id' itself
        dff[col] = dff[col].fillna(dff.groupby('tile_id')[col].transform('median'))
    now_nan = dff[col].isna().sum()
    print(f'reduce nan from {ini_nan} to {now_nan}, ', col)
   
print()
print(len(covs_all))

reduce nan from 484 to 82,  ndti_glad.landast.ard2.seasconv_m_30m_s_YYYY0901_YYYY1031_eu_epsg.3035_v20231127
reduce nan from 4075 to 11,  clm_lst_mod11a2.nighttime_p05_1km_s0..0cm_YYYY.08.01..YYYY.08.31_v1.2
reduce nan from 0 to 0,  clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_YYYY.10.01..YYYY.10.31_v2.1
reduce nan from 482 to 82,  ndwi.gao_glad.landsat.ard2.seasconv.bimonthly.m_30m_s_YYYY0901_YYYY1031_eu_epsg.3035_v20231218
reduce nan from 482 to 82,  ndsi_glad.landast.ard2.seasconv_m_30m_s_YYYY0301_YYYY0430_eu_epsg.3035_v20231127
reduce nan from 482 to 82,  ndwi.gao_glad.landsat.ard2.seasconv.bimonthly.m_30m_s_YYYY1101_YYYY1231_eu_epsg.3035_v20231218
reduce nan from 4075 to 11,  clm_lst_mod11a2.daytime_p95_1km_s0..0cm_YYYY.01.01..YYYY.01.31_v1.2
reduce nan from 4075 to 11,  clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_YYYY.05.01..YYYY.05.31_v1.2
reduce nan from 4075 to 11,  clm_lst_mod11a2.daytime_p05_1km_s0..0cm_YYYY.08.01..YYYY.08.31_v1.2
reduce nan from 482 to 82,  nir_glad.l

In [10]:
import json

with open('cov_all.json', 'w') as file:
    json.dump(covs_all, file, indent=4)

## texture components

In [11]:
for texture_col in ['clay','silt','sand']:
    print(dff[texture_col+'_qa'].unique())

[ 1.  5. nan]
[ 1.  4.  2. nan  5.]
[ 1.  4.  2. nan  5.]


In [12]:
# Round all numeric columns to 1 decimal place
columns_to_round = ['clay', 'silt', 'sand']
columns_round = [i+'_round' for i in columns_to_round]
dff[columns_round] = dff[columns_to_round].round(1)
dff['clay_round'] = np.where(dff['clay_round'] == 0, 0.1, dff['clay_round'])

/tmp/ipykernel_695438/2877583773.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[columns_round] = dff[columns_to_round].round(1)
/tmp/ipykernel_695438/2877583773.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[columns_round] = dff[columns_to_round].round(1)
/tmp/ipykernel_695438/2877583773.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To g

In [13]:
print(dff.shape)
# Calculate the 2 components O1 and O2
dff['texture1'] = dff['sand_round'] / dff['clay_round']
dff['texture2'] = dff['silt_round'] / dff['clay_round']

dff['texture1_qa'] = np.minimum(dff['sand_qa'], dff['clay_qa'])
dff['texture2_qa'] = np.minimum(dff['silt_qa'], dff['clay_qa'])

dff = dff.drop(columns = ['clay_round', 'silt_round', 'sand_round'])
print('data size after adding 2 texture component, ', dff.shape)

(409369, 643)


/tmp/ipykernel_695438/2176668851.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff['texture1'] = dff['sand_round'] / dff['clay_round']
/tmp/ipykernel_695438/2176668851.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff['texture2'] = dff['silt_round'] / dff['clay_round']
/tmp/ipykernel_695438/2176668851.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

data size after adding 2 texture component,  (409369, 644)


In [14]:
dff.to_parquet(f'soil_organized.pq')

### get only the sharable ones

In [15]:
# share = pd.read_csv(f'{folder}/data_share/share_or_not.csv')
# merged_df = pd.merge(df, share, on='ref', how='left')
# share_df = merged_df.loc[merged_df['Can be shared within the consortium']=='Y']

In [16]:
# share_df.to_parquet(f'{folder}/data_share/AI4SH_WP5_soil.property.with.cov.parquet')